In [ ]:
from alphabase.peptide.fragment import create_fragment_mz_dataframe
import pandas as pd

import os

if not os.path.exists('sandbox'):
    os.makedirs('sandbox')

repeat = 10
charged_frag_types = ['b_z1','y_z1','y_modloss_z1']
precursor_df = pd.DataFrame({
    'sequence': ['ASGHCEWMKYR']*repeat+['ASGHCEWMAAR'],
    'mods': ['Acetyl@Protein_N-term;Carbamidomethyl@C;Oxidation@M']*repeat+[''],
    'mod_sites': ['0;4;8']*repeat+[''],
    'nAA': 11,
    'NCE': 20,
    'instrument': 'QE',
    'rt_pred': 10,
    'charge': 2,
    'protein_name': 'unknown',
    'mobility_pred': 1,
})
precursor_df.loc[0,['mods','mod_sites']] = ['Phospho@S','2']
frag_mass_df = create_fragment_mz_dataframe(precursor_df, charged_frag_types)
frag_mass_df

,b_z1,y_z1,y_modloss_z1
0,72.044388,1376.527588,1278.550659
1,239.042755,1209.529175,0.000000
2,296.064209,1152.507690,0.000000
3,433.123138,1015.448792,0.000000
4,536.132324,912.439636,0.000000
...,...,...,...
105,585.208557,634.312988,0.000000
106,771.287903,448.233673,0.000000
107,902.328369,317.193176,0.000000
108,973.365479,246.156067,0.000000


In [ ]:
from alphabase.spectral_library.base import SpecLibBase
spec_lib = SpecLibBase(charged_frag_types)
spec_lib._precursor_df = precursor_df
spec_lib._fragment_intensity_df = frag_mass_df.copy()
spec_lib._fragment_mz_df = frag_mass_df.copy()

In [ ]:
from peptdeep.spec_lib.translate import speclib_to_single_df, translate_to_tsv
import numpy as np
speclib_sdf = speclib_to_single_df(spec_lib)
translate_to_tsv(spec_lib, 'sandbox/lib.tsv', batch_size=2, multiprocessing=True)
ddf = pd.read_csv('sandbox/lib.tsv', sep="\t")
assert len(ddf) == len(speclib_sdf)
assert ddf.StrippedPeptide.values[0] == speclib_sdf.StrippedPeptide.values[0]
assert ddf.StrippedPeptide.values[-1] == speclib_sdf.StrippedPeptide.values[-1]
assert ddf.PrecursorCharge.dtype in [np.int8,np.int32,np.int64]

11it [00:00, 11.84it/s]
100%|██████████| 6/6 [00:00<00:00, 81.63it/s]


Translation finished, it will take several minutes to export the rest precursors to the tsv file...
